Goal of this notebook: given a table-like grid of cells, write an equivalent Python program that can be run on the input data (cells without any dependencies) to generate the output (cells in the last column or cells without any dependencies).

First example: one single row.

In [ ]:
def translate(table):
    ...
    
table = [['3', '4', '=A1+B1']]

# Equivalent program:
def f_2(x, y):
    return x + y  # Translation of 'A1+B1'

def row_function(table):
    ...

def input_columns(table):
    ...

def output_columns(table):
    ...
    
row_function(table) == f_2  # More like AST of f_2 but w/e
input_columns(table) == [0, 1]
output_columns(table) == [2]

# To evaluate:
# for row in table:
#    table[output_columns] = row_function(table)(*[row[col] for col in input_columns(table)]))
None

In [ ]:
from excelbutbetter.parser import parse

No js module found, not running main scripts.


In [ ]:
ast = parse('=A1+B1')
ast

InfixOp(op='\\+', left=Ref(row=0, column=0, fixed_row=False, fixed_column=False), right=Ref(row=0, column=1, fixed_row=False, fixed_column=False))

The expected result here is f(x, y) = x + y. Let's build up code bottom-up.

In [ ]:
infix_translate_map = {
    '\\+': lambda x, y: f'{translate(x)} + {translate(y)}'
}

prefix_translate_map = {}
postfix_translate_map = {}

In [ ]:
def translate(tree, context=None):  # Todo(Rik): str might translate to ref.
    if isinstance(tree, (int, float, bool, str)):
        return tree
    elif isinstance(tree, InfixOp) and tree.op == ':':
        # Todo(Rik): handle ranges in formulas
        raise NotImplementedError
    elif isinstance(tree, InfixOp):
        return infix_translate_map[tree.op](translate(tree.left, context), translate(tree.right, context))
    elif isinstance(tree, PrefixOp):
        return prefix_translate_map[tree.op](translate(tree.arg, context))
    elif isinstance(tree, PostfixOp):
        return postfix_translate_map[tree.op](translate(tree.arg, context))
    elif isinstance(tree, Function):
#         if tree.args is None: return function_translate_map[tree.name]()
        args = translate(tree.args, context)
        if not isinstance(args, tuple): # Gymnastics to handle one-argument functions
            args = (args,)
        return function_translate_map[tree.name](args)
    elif isinstance(tree, Ref):  # Responsiblity of the caller to handle IndexError
        return context[tree.row][tree.column]

In [ ]:
translate(ast)

NameError: name 'InfixOp' is not defined